In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.0.0'

In [ ]:
df = pd.read_excel('tests/test_data/data_with_multiplex.xlsx')

In [ ]:
df

In [ ]:
groups = df.groupby(['From', 'To'])

In [ ]:
groups.all()

In [ ]:
groups = df.groupby(['To', 'From'])

In [ ]:
groups.all()

We can carve the data by To and by From. We will do both, then figure out which combination of groupings results in the fewest total queries.

We should be aiming for

- (Bath, Bristol) x London
- Edinburgh x (Bath, Birmingham)
- Darlington x Greenwich


In [ ]:
subgroups = df.groupby(['From', 'To']).all()


In [ ]:
subgroups

We have eliminated one row, but we have duplicate `To`.

In [ ]:
subgroups2 = subgroups.groupby(['To', 'From']).all()
subgroups2

Now we perform an inner merge.

In [ ]:
out = pd.merge(subgroups, subgroups2, left_on='From', right_on='From', how='inner')

In [ ]:
out

In [ ]:
df.merge(df, left_on='From', right_on='From', how='left')

In [ ]:
df.merge(df, left_on='To', right_on='To', how='inner')

In [ ]:
a = df.groupby(['From'])

In [ ]:
a2 = a['To'].unique()

In [ ]:
b = df.groupby(['To'])

In [ ]:
b2 = b['From'].unique()

In [ ]:
a3 = a2.reset_index()

In [ ]:
a3

In [ ]:
b3 = b2.reset_index()

In [ ]:
b3['To']

In [ ]:
a3

In [ ]:
a3.iloc[3]['To']

In [ ]:
a3['To'] = a3['To'].apply(lambda x: x[0] if len(x) == 1 else x)

In [ ]:
a3

In [ ]:
b3['To'] = b3['To'].apply(lambda x: x[0] if len(x) == 1 else x)

In [ ]:
pd.merge(a3, b3)

In [ ]:
a3.groupby('To')

In [ ]:
a3.groupby('To')

In [ ]:
a4 = a3.groupby('To')

In [ ]:
gb = df.groupby(['From', 'To'])['To'].unique()

In [ ]:
gb

In [ ]:
gb = df.groupby(['From', 'To'])['From'].unique()

In [ ]:
gb

In [ ]:
gb

In [ ]:
a2.reset_index()

In [ ]:
b2.reset_index()

In [ ]:
df

Drop single elements from the groupbys, we will add them again later

In [ ]:
a3

In [ ]:
multiple_records = a3.apply(lambda x: len(x['To']) > 1, axis=1)

In [ ]:
a3 = a2.reset_index()

In [ ]:
a4 = a3.loc[multiple_records]

In [ ]:
a4

In [3]:
def filter_for_multiples(df, key='From'):
    multiple_records = df.apply(lambda x: len(x[key]) > 1, axis=1)
    
    return df.loc[multiple_records]



In [ ]:
b4 = filter_for_multiples(b2.reset_index())

In [ ]:
a4

In [ ]:
b3

In [ ]:
_from = 'Edinburgh, UK'

In [ ]:
_from in a4['From'].values

In [ ]:
out = []
for _, i in b3.iterrows():
    
    if len(i['From']) == 1:
        _from = i['From'][0]
        if _from in a4['From'].values:
            # We already have it
            print(_from)
            pass
        else:
            # Not part of the other groupby 
            out.append(i)
            
    else:
        # It's a group, we definitely want it
        out.append(i)
            

In [ ]:
out

In [ ]:
a4.append(out)

In [ ]:
a = df.groupby(['From'])['To'].unique().reset_index()

In [ ]:
a

In [18]:
def factorize_locations(df):
    gb_from = df.groupby(['From'])['To'].unique().reset_index()
    gb_to = df.groupby(['To'])['From'].unique().reset_index()
    
    # Drop all non-grouped locations from gb_from
    gb_from = filter_for_multiples(gb_from, key='To')
    
    # Now figure out which ones we want from gb_to
    out = []
    for _, i in gb_to.iterrows():
        if len(i['From']) == 1:
            _from = i['From'][0]
            if _from in gb_from['From'].values:
                # We already have it 
                pass
            else:
                # Not part of the other groupby 
                out.append(i)
        else:
            # It's a group, we definitely want it
            out.append(i)

    factorized_df = gb_from.append(out)

    # Check that we're not missing anything
    assert len(factorized_df.explode('From').explode('To')) == len(df)
    
    return factorized_df

In [19]:
factorize_locations(df)

,From,To
0,"Bagshot, UK","[Bristol, UK, Leeds, UK]"
4,"Edinburgh, UK","[Bath, UK, Birmingham, UK, Bagshot, UK]"
4,"[Darlington, UK]","Greenwich, UK"
6,"[Bristol, UK, Bath, UK]","London, UK"


In [20]:
df = pd.read_excel('tests/test_data/data_with_multiplex_big.xlsx')
factorized_df = factorize_locations(df)

In [21]:
df

,From,To
0,"Bristol, UK","London, UK"
1,"Bath, UK","London, UK"
2,"Edinburgh, UK","Bath, UK"
3,"Edinburgh, UK","Birmingham, UK"
4,"Darlington, UK","Greenwich, UK"
5,"Edinburgh, UK","Bagshot, UK"
6,"Bagshot, UK","Bristol, UK"
7,"Bagshot, UK","Leeds, UK"


In [7]:
factorized_df

,From,To
0,"Bagshot, UK","[Bristol, UK, Leeds, UK]"
4,"Edinburgh, UK","[Bath, UK, Birmingham, UK, Bagshot, UK]"
4,"[Darlington, UK]","Greenwich, UK"
6,"[Bristol, UK, Bath, UK]","London, UK"


In [11]:
factorized_df.explode('From').explode('To')

,From,To
0,"Bagshot, UK","Bristol, UK"
0,"Bagshot, UK","Leeds, UK"
4,"Edinburgh, UK","Bath, UK"
4,"Edinburgh, UK","Birmingham, UK"
4,"Edinburgh, UK","Bagshot, UK"
4,"Darlington, UK","Greenwich, UK"
6,"Bristol, UK","London, UK"
6,"Bath, UK","London, UK"


In [12]:
assert len(factorized_df.explode('From').explode('To')) == len(df)

In [22]:
row = factorized_df.iloc[0]

In [32]:
pd.DataFrame(row).T.explode('From').explode('To')

,From,To
0,"Bagshot, UK","Bristol, UK"
0,"Bagshot, UK","Leeds, UK"
